# Isozymes in yeast

This notebook contains a preliminary study of isozymes in _S. cerevisiae_.

Benjamín J. Sánchez, 2020-08-16

## 1. Loading Model and Data

In [1]:
import cobra
import os
import pandas as pd
import wget

wget.download("https://raw.githubusercontent.com/SysBioChalmers/ecModels/master/ecYeastGEM/model/ecYeastGEM.xml",
              "ecYeastGEM.xml", bar=False)
model = cobra.io.read_sbml_model("ecYeastGEM.xml")
os.remove("ecYeastGEM.xml")

Using license file C:\Users\bejsab\gurobi.lic
Academic license - for non-commercial use only


In [2]:
# proteomics data:
df = pd.read_csv("1710_mq14_sample_default_settings_proteinGroups.txt", sep="\t", index_col=0)
print(df)

                                                                                 Majority protein IDs  \
Protein IDs                                                                                             
A5Z2X5                                                                                         A5Z2X5   
CON__P02533                                                                               CON__P02533   
CON__P04259;CON__P02538;CON__P48668;CON__tr|A4F...  CON__P04259;CON__P02538;CON__P48668;CON__tr|A4...   
CON__P04264;CON__tr|G3N0V2|G3N0V2_BOVIN                                                   CON__P04264   
CON__P08779;CON__Q9Z2K1;CON__Q3ZAW8                                                       CON__P08779   
...                                                                                               ...   
P40207                                                                                         P40207   
P40208                                                 

## 2. Isozyme occurrence

In [3]:
isozymes_in_model = 0
for reaction in model.reactions:
    if "arm_" in reaction.id and "_REV" not in reaction.id:
        isozymes_in_model += 1
print("Number of reactions in the model with isozymes: " + str(isozymes_in_model))

Number of reactions in the model with isozymes: 445


In [4]:
# find enzyme from id:
def find_protein_id(gene):
    for reaction in gene.reactions:
        if "_exchange" in reaction.id:
            return reaction.id.replace("prot_","").replace("_exchange","")

# count isozymes in data:
isozymes_in_data = []
for gene_A in model.genes:
    try:
        protein_A = find_protein_id(gene_A)
        if protein_A in df.index:
            for gene_B in model.genes:
                if gene_A != gene_B:
                    try:
                        protein_B = find_protein_id(gene_B)
                        if protein_B in df.index:
                            for reaction_A in gene_A.reactions:
                                if "arm_" in reaction_A.id and "_REV" not in reaction_A.id:
                                    rule = reaction_A.gene_reaction_rule
                                    if gene_B.id in rule and not " and " in rule:
                                        isozymes_in_data.append([protein_A, protein_B])
                                        break
                    except:
                        pass
    except:
        pass
print("Number of reactions with isozymes: " + str(len(isozymes_in_data)/2))  # Every occurrence is counted twice

Number of reactions with isozymes: 47.0
